In [18]:
import os

os.environ["GIT_CLONE_PROTECTION_ACTIVE"] = "false"

In [20]:
%pip install optimum


Note: you may need to restart the kernel to use updated packages.


In [21]:
%pip install optimum-intel



  Using cached optimum_intel-1.18.0-py3-none-any.whl.metadata (16 kB)
Using cached optimum_intel-1.18.0-py3-none-any.whl (223 kB)


In [22]:
%pip install -Uq pip

Note: you may need to restart the kernel to use updated packages.


In [23]:
%pip uninstall -q -y optimum optimum-intel

Note: you may need to restart the kernel to use updated packages.


In [24]:
%pip install --pre -Uq openvino openvino-tokenizers[transformers] --extra-index-url https://storage.openvinotoolkit.org/simple/wheels/nightly
%pip install -q --extra-index-url https://download.pytorch.org/whl/cpu\
"git+https://github.com/huggingface/optimum-intel.git"\
"git+https://github.com/openvinotoolkit/nncf.git"\
"torch>=2.1"\
"datasets" \
"accelerate"\
"gradio>=4.19"\
"onnx" "einops" "transformers_stream_generator" "tiktoken" "transformers>=4.40" "bitsandbytes"

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [25]:
import os
from pathlib import Path
import requests
import shutil


In [26]:
config_shared_path = Path("../../utils/llm_config.py")
config_dst_path = Path("llm_config.py")

if not config_dst_path.exists():
    if config_shared_path.exists():
        try:
            os.symlink(config_shared_path, config_dst_path)
        except Exception:
            shutil.copy(config_shared_path, config_dst_path)
    else:
        r = requests.get(url="https://raw.githubusercontent.com/openvinotoolkit/openvino_notebooks/latest/utils/llm_config.py")
        with open("llm_config.py", "w", encoding="utf-8") as f:
            f.write(r.text)
elif not os.path.islink(config_dst_path):
    print("LLM config will be updated")
    if config_shared_path.exists():
        shutil.copy(config_shared_path, config_dst_path)
    else:
        r = requests.get(url="https://raw.githubusercontent.com/openvinotoolkit/openvino_notebooks/latest/utils/llm_config.py")
        with open("llm_config.py", "w", encoding="utf-8") as f:
            f.write(r.text)

LLM config will be updated


In [27]:
from huggingface_hub import notebook_login, whoami

try:
    whoami()
    print('Authorization token already provided')
except OSError:
    notebook_login()


Authorization token already provided


In [28]:
from llm_config import SUPPORTED_LLM_MODELS
import ipywidgets as widgets

In [29]:
model_languages = list(SUPPORTED_LLM_MODELS)

model_language = widgets.Dropdown(
    options=model_languages,
    value=model_languages[0],
    description="Model Language:",
    disabled=False,
)

model_language

Dropdown(description='Model Language:', options=('English', 'Chinese', 'Japanese'), value='English')

In [30]:
model_ids = list(SUPPORTED_LLM_MODELS[model_language.value])

model_id = widgets.Dropdown(
    options=model_ids,
    value=model_ids[0],
    description="Model:",
    disabled=False,
)

model_id

Dropdown(description='Model:', options=('qwen2-0.5b-instruct', 'tiny-llama-1b-chat', 'qwen2-1.5b-instruct', 'g…

In [31]:
model_configuration = SUPPORTED_LLM_MODELS[model_language.value][model_id.value]
print(f"Selected model {model_id.value}")

Selected model llama-3-8b-instruct


In [32]:
from IPython.display import Markdown, display

prepare_int4_model = widgets.Checkbox(
    value=True,
    description="Prepare INT4 model",
    disabled=False,
)
prepare_int8_model = widgets.Checkbox(
    value=False,
    description="Prepare INT8 model",
    disabled=False,
)
prepare_fp16_model = widgets.Checkbox(
    value=False,
    description="Prepare FP16 model",
    disabled=False,
)

display(prepare_int4_model)
display(prepare_int8_model)
display(prepare_fp16_model)

Checkbox(value=True, description='Prepare INT4 model')

Checkbox(value=False, description='Prepare INT8 model')

Checkbox(value=False, description='Prepare FP16 model')

In [33]:
enable_awq = widgets.Checkbox(
    value=False,
    description="Enable AWQ",
    disabled=not prepare_int4_model.value,
)
display(enable_awq)

Checkbox(value=False, description='Enable AWQ')

In [34]:
from pathlib import Path

pt_model_id = model_configuration["model_id"]
pt_model_name = model_id.value.split("-")[0]
fp16_model_dir = Path(model_id.value) / "FP16"
int8_model_dir = Path(model_id.value) / "INT8_compressed_weights"
int4_model_dir = Path(model_id.value) / "INT4_compressed_weights"


def convert_to_fp16():
    if (fp16_model_dir / "openvino_model.xml").exists():
        return
    remote_code = model_configuration.get("remote_code", False)
    export_command_base = "optimum-cli export openvino --model {} --task text-generation-with-past --weight-format fp16".format(pt_model_id)
    if remote_code:
        export_command_base += " --trust-remote-code"
    export_command = export_command_base + " " + str(fp16_model_dir)
    display(Markdown("**Export command:**"))
    display(Markdown(f"`{export_command}`"))
    ! $export_command


def convert_to_int8():
    if (int8_model_dir / "openvino_model.xml").exists():
        return
    int8_model_dir.mkdir(parents=True, exist_ok=True)
    remote_code = model_configuration.get("remote_code", False)
    export_command_base = "optimum-cli export openvino --model {} --task text-generation-with-past --weight-format int8".format(pt_model_id)
    if remote_code:
        export_command_base += " --trust-remote-code"
    export_command = export_command_base + " " + str(int8_model_dir)
    display(Markdown("**Export command:**"))
    display(Markdown(f"`{export_command}`"))
    ! $export_command


def convert_to_int4():
    compression_configs = {
        "zephyr-7b-beta": {
            "sym": True,
            "group_size": 64,
            "ratio": 0.6,
        },
        "mistral-7b": {
            "sym": True,
            "group_size": 64,
            "ratio": 0.6,
        },
        "minicpm-2b-dpo": {
            "sym": True,
            "group_size": 64,
            "ratio": 0.6,
        },
        "gemma-2b-it": {
            "sym": True,
            "group_size": 64,
            "ratio": 0.6,
        },
        "notus-7b-v1": {
            "sym": True,
            "group_size": 64,
            "ratio": 0.6,
        },
        "neural-chat-7b-v3-1": {
            "sym": True,
            "group_size": 64,
            "ratio": 0.6,
        },
        "llama-2-chat-7b": {
            "sym": True,
            "group_size": 128,
            "ratio": 0.8,
        },
        "llama-3-8b-instruct": {
            "sym": True,
            "group_size": 128,
            "ratio": 0.8,
        },
        "gemma-7b-it": {
            "sym": True,
            "group_size": 128,
            "ratio": 0.8,
        },
        "chatglm2-6b": {
            "sym": True,
            "group_size": 128,
            "ratio": 0.72,
        },
        "qwen-7b-chat": {"sym": True, "group_size": 128, "ratio": 0.6},
        "red-pajama-3b-chat": {
            "sym": False,
            "group_size": 128,
            "ratio": 0.5,
        },
        "default": {
            "sym": False,
            "group_size": 128,
            "ratio": 0.8,
        },
    }

    model_compression_params = compression_configs.get(model_id.value, compression_configs["default"])
    if (int4_model_dir / "openvino_model.xml").exists():
        return
    remote_code = model_configuration.get("remote_code", False)
    export_command_base = "optimum-cli export openvino --model {} --task text-generation-with-past --weight-format int4".format(pt_model_id)
    int4_compression_args = " --group-size {} --ratio {}".format(model_compression_params["group_size"], model_compression_params["ratio"])
    if model_compression_params["sym"]:
        int4_compression_args += " --sym"
    if enable_awq.value:
        int4_compression_args += " --awq --dataset wikitext2 --num-samples 128"
    export_command_base += int4_compression_args
    if remote_code:
        export_command_base += " --trust-remote-code"
    export_command = export_command_base + " " + str(int4_model_dir)
    display(Markdown("**Export command:**"))
    display(Markdown(f"`{export_command}`"))
    ! $export_command


if prepare_fp16_model.value:
    convert_to_fp16()
if prepare_int8_model.value:
    convert_to_int8()
if prepare_int4_model.value:
    convert_to_int4()

In [35]:
fp16_weights = fp16_model_dir / "openvino_model.bin"
int8_weights = int8_model_dir / "openvino_model.bin"
int4_weights = int4_model_dir / "openvino_model.bin"

if fp16_weights.exists():
    print(f"Size of FP16 model is {fp16_weights.stat().st_size / 1024 / 1024:.2f} MB")
for precision, compressed_weights in zip([8, 4], [int8_weights, int4_weights]):
    if compressed_weights.exists():
        print(f"Size of model with INT{precision} compressed weights is {compressed_weights.stat().st_size / 1024 / 1024:.2f} MB")
    if compressed_weights.exists() and fp16_weights.exists():
        print(f"Compression rate for INT{precision} model: {fp16_weights.stat().st_size / compressed_weights.stat().st_size:.3f}")

Size of model with INT4 compressed weights is 5085.79 MB


In [36]:
import openvino as ov

core = ov.Core()

support_devices = core.available_devices
if "NPU" in support_devices:
    support_devices.remove("NPU")

device = widgets.Dropdown(
    options=support_devices + ["AUTO"],
    value="CPU",
    description="Device:",
    disabled=False,
)

device

Dropdown(description='Device:', options=('CPU', 'GPU', 'AUTO'), value='CPU')

In [37]:
available_models = []
if int4_model_dir.exists():
    available_models.append("INT4")
if int8_model_dir.exists():
    available_models.append("INT8")
if fp16_model_dir.exists():
    available_models.append("FP16")

model_to_run = widgets.Dropdown(
    options=available_models,
    value=available_models[0],
    description="Model to run:",
    disabled=False,
)

model_to_run

Dropdown(description='Model to run:', options=('INT4',), value='INT4')

In [38]:
from transformers import AutoConfig, AutoTokenizer
from optimum.intel.openvino import OVModelForCausalLM

if model_to_run.value == "INT4":
    model_dir = int4_model_dir
elif model_to_run.value == "INT8":
    model_dir = int8_model_dir
else:
    model_dir = fp16_model_dir
print(f"Loading model from {model_dir}")

ov_config = {"PERFORMANCE_HINT": "LATENCY", "NUM_STREAMS": "1", "CACHE_DIR": ""}

if "GPU" in device.value and "qwen2-7b-instruct" in model_id.value:
    ov_config["GPU_ENABLE_SDPA_OPTIMIZATION"] = "NO"


if model_id.value == "red-pajama-3b-chat" and "GPU" in core.available_devices and device.value in ["GPU", "AUTO"]:
    ov_config["INFERENCE_PRECISION_HINT"] = "f32"

model_name = model_configuration["model_id"]
tok = AutoTokenizer.from_pretrained(model_dir, trust_remote_code=True)

ov_model = OVModelForCausalLM.from_pretrained(
    model_dir,
    device=device.value,
    ov_config=ov_config,
    config=AutoConfig.from_pretrained(model_dir, trust_remote_code=True),
    trust_remote_code=True,
)

Loading model from llama-3-8b-instruct\INT4_compressed_weights


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
The argument `trust_remote_code` is to be used along with export=True. It will be ignored.
Compiling the model to CPU ...


In [ ]:
tokenizer_kwargs = model_configuration.get("tokenizer_kwargs", {})
test_string = "10+20 ="
input_tokens = tok(test_string, return_tensors="pt", **tokenizer_kwargs)
answer = ov_model.generate(**input_tokens, max_new_tokens=2)
print(tok.batch_decode(answer, skip_special_tokens=True)[0])

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [15]:
pip install gradio transformers torch


Note: you may need to restart the kernel to use updated packages.


In [4]:
import gradio as gr
from uuid import uuid4

gitam_examples = [
    "What are the different undergraduate programs offered at GITAM University?",
    "How can I apply for GITAM admission?",
    "Can you tell me about the campus facilities?",
    "What is the fee structure at GITAM University?",
    "Do you offer any scholarships?",
    "Where can I find more information on research opportunities?",
    "How do I contact the International Affairs Office?",
    "Can you provide information on student housing?",
    "Tell me about hostel life at GITAM.",
    "What are the important dates for admission?",
    "What are the Ph.D. programs available?",
    "What is the admission process for international students?",
    "How do I contact the university?"
]
keyword_categories = {
    "Library": ["library resources", "study materials", "digital library", "book lending", "research support", "reading rooms", "library services", "academic journals", "library hours", "e-books", "online databases", "library catalog", "reference services", "library facilities"],
    "Placements": ["placements", "placement assistance", "career development", "career guidance", "job opportunities", "employment support"],
    "Transport Facilities": ["transport facilities", "bus service", "shuttle service", "transportation options", "commuting options", "campus transport", "transportation facilities"],
    "Safety & Security": ["safety measures", "security measures", "campus security", "security services", "emergency procedures", "safety protocols"],
    "Apply": ["apply for admission", "application process", "admission requirements", "online application", "submit application", "application form", "application deadline", "admission procedure"],
    "Gitam School of Architecture": ["architecture program", "BArch", "M.Arch", "architecture courses", "design studio", "architectural theory", "building construction", "urban planning", "landscape architecture"],
    "Application/Documents": ["application process", "required documents", "admission forms", "submission deadlines", "application status", "transcripts", "recommendation letters", "personal statement"],
    "Hostel Life": ["hostel", "hostels", "accommodation", "residence halls", "living on campus", "roommates", "facilities", "food options", "rules and regulations"],
    "fee": ["fee", "fees", "refund"],
    "scholarship": ["scholarship", "scholarships", "financial aid"],
    "international_affairs": ["international students", "global programs", "study abroad", "visa assistance", "cross-cultural support"],
    "academic": ["programs", "courses", "degrees", "disciplines", "undergraduate", "postgraduate", "ph.d.", "doctorate"],
    "application": ["apply", "admission", "admissions", "application"],
    "campus": ["campus", "facilities", "location", "hostel", "housing", "life"],
    "faculty": ["faculty", "professors", "teachers"],
    "learning": ["learning", "courses", "online"],
    "career": ["placement", "job", "career"],
    "life": ["life", "housing", "sports", "dining", "security", "transport"],
    "research": ["research", "projects", "PhD", "doctorate"],
    "entrepreneurship": ["entrepreneurship", "startup", "business"],
    "accreditation": ["accreditation", "rankings", "recognition"],
    "alumni": ["alumni", "graduates", "network"],
    "parents": ["parents", "guardian", "family"],
    "leadership": ["leadership", "administration", "staff"],
    "dates": ["dates", "schedule", "calendar"],
    "contact": ["contact", "phone", "email"],
    "Postgraduate": {
        "architecture": ["M.Arch", "Master of Architecture", "architecture"],
        "engineering": ["M.Tech", "Master of Technology", "engineering"],
        "humanities": ["MA", "Master of Arts", "humanities"],
        "law": ["LLM", "Master of Laws", "law"],
        "management": ["MBA", "Master of Business Administration", "management"],
        "medicine": ["MD", "Doctor of Medicine", "MS", "Master of Surgery", "medicine"],
        "nursing": ["MSc Nursing", "Master of Science in Nursing", "nursing"],
        "paramedical": ["MPT", "Master of Physiotherapy", "MSc MLT", "Master of Science in Medical Laboratory Technology", "paramedical"],
        "pharmacy": ["MPharm", "Master of Pharmacy", "PharmD", "Doctor of Pharmacy", "pharmacy"],
        "physiotherapy": ["MPT", "Master of Physiotherapy", "physiotherapy"],
        "public_policy": ["MPP", "Master of Public Policy", "public policy"],
        "science": ["MSc", "Master of Science", "science"]
    },
    "Undergraduate": {
        "BTech": {
            "CSE": ["Computer Science and Engineering", "Artificial Intelligence and Machine Learning (AIML)", "Data Science", "Cyber Security", "Internet of Things (IoT)", "Software Engineering"],
            "ECE": ["Electronics and Communication Engineering", "Embedded Systems", "VLSI Design", "Signal Processing", "Telecommunications"],
            "EEE": ["Electrical and Electronics Engineering", "Power Systems", "Control Systems", "Renewable Energy"],
            "ME": ["Mechanical Engineering", "Automobile Engineering", "Robotics", "Thermal Engineering", "Manufacturing"],
            "CE": ["Civil Engineering", "Structural Engineering", "Construction Management", "Environmental Engineering"],
            "IT": ["Information Technology", "Network Security", "Cloud Computing", "Web Technologies"],
            "Biotech": ["Biotechnology", "Biochemical Engineering", "Genetic Engineering"],
            "Chemical": ["Chemical Engineering", "Process Engineering", "Petrochemical Engineering"]
        },
        "Architecture": ["BArch", "Bachelor of Architecture"],
        "Engineering": ["BEng", "Bachelor of Engineering in various specializations"],
        "Humanities": ["BA", "Bachelor of Arts in various fields such as English, Economics, Psychology"],
        "Law": ["LLB", "Bachelor of Laws", "BA LLB", "Integrated Bachelor of Arts and Bachelor of Laws"],
        "Management": ["BBA", "Bachelor of Business Administration", "BCom", "Bachelor of Commerce"],
        "Medicine": ["MBBS", "Bachelor of Medicine, Bachelor of Surgery"],
        "Nursing": ["BSc Nursing", "Bachelor of Science in Nursing"],
        "Paramedical": ["BPT", "Bachelor of Physiotherapy", "BSc MLT", "Bachelor of Science in Medical Laboratory Technology"],
        "Pharmacy": ["BPharm", "Bachelor of Pharmacy", "PharmD", "Doctor of Pharmacy"],
        "Physiotherapy": ["BPT", "Bachelor of Physiotherapy"],
        "Public Policy": ["BPP", "Bachelor of Public Policy"],
        "Science": ["BSc", "Bachelor of Science in various fields such as Physics, Chemistry, Mathematics, Biology"]
    }
}

responses = {
    "Library":"Explore Gitam University's library offerings, featuring extensive digital collections, academic journals, e-books, and online databases tailored to support your research and study needs. Visit https://library.gitam.edu/ to learn more.",
    "Placements": "At GITAM University, our Career Development Center is committed to enhancing both personal and professional skills, ensuring that students are fully prepared for successful careers. With 100% placement assistance, we aim to support students in reaching their career aspirations. For more information on our placement initiatives, please visit [GITAM Career Guidance Center](https://www.gitam.edu/gitam-career-guidance-center).",
    "Transport Facilities": "For your convenience, Gitam campuses provide extensive transport facilities including bus and shuttle services. These services ensure easy commuting for students and staff within and around the campus. For more details, visit https://www.gitam.edu/campus-life/transport-facilities.",
    "Safety & Security": "At Gitam, your safety is our priority. We have implemented robust safety measures and security arrangements, including comprehensive emergency protocols and surveillance systems, to ensure a secure environment for our students. Learn more at https://www.gitam.edu/campus-life/safety-security.",
    "Apply": "For the Gitam Admission Test (GAT), applicants can apply by following these steps: register for the exam, choose their exam date, time, and centre, download their admit card, take the GAT exam, check their results, and attend counselling if they pass. Merit scholarships, awarded based on GAT scores, can cover up to 100% of the first-year fee and are renewable annually with a CGPA of 8.0 or higher. Apply at https://apply.gitam.edu/.",
    "Application/Documents": "Applying for the GAT/GBAT at GITAM involves exploring program options, registering securely online, ensuring all required documents are gathered, verifying details thoroughly, and printing the completed form after payment.",
    "Hostel Life": "Gitam University's hostels offer comfortable accommodation equipped with amenities such as mess facilities, Wi-Fi access, and recreational areas, fostering a homely environment for students. For more details, visit the hostel facilities page on the Gitam website: https://www.gitam.edu/campus-life/residential-life.",
    "fee": "For detailed fee structure information at GITAM University, visit their official admissions page at https://www.gitam.edu/admissions/fee-structure. This page provides comprehensive details on fees for various programs, including scholarships and payment schedules, making it the best resource for understanding financial requirements for your education at GITAM.",
    "scholarship": "GITAM University offers various scholarships, including financial aid, merit scholarships for academic and extracurricular excellence, and sports scholarships to support student athletes. For more details, visit https://www.gitam.edu/admissions/student-scholarships.",
    "international_affairs": "GITAM University's dedicated International Affairs Office supports international students with admissions, visa processes, and comprehensive assistance services. For more information, visit the international students' page on the Gitam website: https://www.gitam.edu/directorate-of-international-affairs.",
    "academic":"GITAM University offers a wide range of undergraduate, postgraduate, and doctoral programs spanning diverse disciplines. For more information, visit the academic programs page on the official website: https://www.gitam.edu/academics.",
    "application": "To apply for admission, visit the Gitam University admissions page. You'll find detailed information on the application process, eligibility criteria, and important deadlines.",
    "campus": "GITAM University's campuses are equipped with state-of-the-art facilities including libraries, laboratories, sports complexes, and more. For more details, visit the campus facilities page.",
    "faculty": "GITAM University boasts a team of highly qualified and experienced faculty members who are dedicated to providing quality education. For more information, visit the faculty page on the official website.",
    "learning": "GITAM University offers a variety of learning opportunities including online courses, workshops, and seminars to enhance students' knowledge and skills. For more details, visit the learning resources page.",
    "career": "GITAM University's Career Development Center provides placement assistance, career guidance, and job opportunities to help students achieve their professional goals. For more information, visit the career services page.",
    "life": "Campus life at GITAM University is vibrant and diverse, with numerous activities, events, and organizations to engage students. For more details, visit the campus life page.",
    "research": "GITAM University offers extensive research opportunities across various disciplines. For more details, visit the research programs page on the https://www.gitam.edu/research.",
    "entrepreneurship": "GITAM University supports entrepreneurship through various initiatives and programs to nurture startup ideas and business ventures. For more details, visit the entrepreneurship center page  https://gventurefunds.gitam.edu/home/.",
    "accreditation": "GITAM University is accredited and recognized by various national and international bodies. For more details, visit the accreditation and rankings page  https://www.gitam.edu/about/accreditation.",
    "alumni": "GITAM University boasts a robust network of alumni who have excelled in diverse fields, contributing significantly to their professions and communities. For detailed information, please explore the alumni page on the official website: https://alumni.gitam.edu/. Learn about their inspiring journeys and accomplishments, and discover how they continue to make a positive impact globally.",
    "parents": "GITAM University provides various resources and support services for parents and guardians. For more details, visit the parents' information page  https://gparent.gitam.edu/login.",
    "leadership": "GITAM University, founded by M.V.V.S Murthi and currently led by M.Bharat as President, Virender Singh Chauhan as Chancellor, Mathukumilli Bharadwaj as Secretary, and Dayananda Siddavattam as Vice-Chancellor, is guided by a team of experienced administrators and staff. For more details, visit the leadership page on the official website: https://www.gitam.edu/about/leadership.",
    "Undergraduate": "GITAM University offers a diverse range of undergraduate programs across various disciplines. In engineering, you can choose from specialties like Computer Science and Engineering (including AI and Machine Learning, Data Science, Cyber Security), Electronics and Communication Engineering, Electrical and Electronics Engineering, Mechanical Engineering, Civil Engineering, and more. Additionally, we provide programs in Architecture, Humanities, Law, Management, Medicine, Nursing, Paramedical studies, Pharmacy, Physiotherapy, Public Policy, and various fields of Science. Explore our offerings to find the perfect fit for your academic and career goals!",
    "dates": "The schedule for admissions includes the following dates: the last date to apply is 14th June 2024, the examination is scheduled for 16th June 2024, personal interviews will be conducted from 17th to 19th June 2024, and results will be announced on 17th June 2024.",
    "contact":"For any inquiries or support, you can contact GITAM University at 08912790101 or via email. Feel free to visit the contact page at [https://www.gitam.edu/contact-us](https://www.gitam.edu/contact-us) for more detailed information and assistance.",
}
def categorize_query(user_message):
    user_message_lower = user_message.lower()
    for category, keywords in keyword_categories.items():
        if any(keyword in user_message_lower for keyword in keywords):
            return category
    return None

def generate_response(category):
    if category in responses:
        return responses[category]
    else:
        return "Please ask relevant details about GITAM University."

def user(message, history):
    history = history + [[message, ""]]
    return "", history

def bot(history, temperature, top_p, top_k, repetition_penalty, conversation_id):
    user_message = history[-1][0]
    category = categorize_query(user_message)
    
    if category:
        response = generate_response(category)
    else:
        response = "Please ask relevant details about GITAM University."
    
    # Update the last user message with the bot's response
    history[-1][1] = response
    yield history

def request_cancel():
    return None

with gr.Blocks() as demo:
    conversation_id = gr.State(str(uuid4()))

    gr.Markdown("""
        <div style="background-color: #f0f8ff; padding: 20px; border-radius: 10px; display: flex; align-items: center;">
            <div style="flex: 1;">
                <h1>Gitam University Query Assistant</h1>
                <p>Ask anything about GITAM University and get instant answers.</p>
            </div>
            <img src="https://i.postimg.cc/pXqxv07h/Gitam-New-Logo.jpg" alt="Gitam University Logo" style="height: 100px; margin-left: 20px;">
        </div>
    """)
    chatbot = gr.Chatbot(value=[["Hello, Welcome to Gitam University. How can I help you?", ""]])
    with gr.Row():
        msg = gr.Textbox(
            show_label=False,
            placeholder="Enter your query and press enter.",
        )
        submit = gr.Button("Submit")
    
    with gr.Row():
        stop = gr.Button("Stop")
        clear = gr.Button("Clear")
    
    with gr.Row():
        with gr.Column():
            temperature = gr.Slider(
                label="Temperature",
                value=0.9,
                minimum=0.1,
                maximum=1.0,
                step=0.1,
                interactive=True,
                info="Randomness of the response — higher values mean more random responses.",
            )
        with gr.Column():
            top_p = gr.Slider(
                label="Top-p (nucleus sampling)",
                value=0.9,
                minimum=0.05,
                maximum=1.0,
                step=0.05,
                interactive=True,
                info="Sample from the top-p percentage of most likely tokens.",
            )
        with gr.Column():
            top_k = gr.Slider(
                label="Top-k",
                value=50,
                minimum=1,
                maximum=100,
                step=1,
                interactive=True,
                info="Sample from a shortlist of top-k tokens — 0 to disable and sample from all tokens.",
            )
        with gr.Column():
            repetition_penalty = gr.Slider(
                label="Repetition Penalty",
                value=1.1,
                minimum=1.0,
                maximum=2.0,
                step=0.1,
                interactive=True,
                info="Penalize repetition — 1.0 to disable.",
            )
    
    gr.Examples(gitam_examples, inputs=msg, label="Click on any example and press the 'Submit' button")
    
    submit_event = msg.submit(
        fn=user,
        inputs=[msg, chatbot],
        outputs=[msg, chatbot],
        queue=False,
    ).then(
        fn=bot,
        inputs=[
            chatbot,
            temperature,
            top_p,
            top_k,
            repetition_penalty,
            conversation_id,
        ],
        outputs=chatbot,
        queue=True,
    )
    
    submit_click_event = submit.click(
        fn=user,
        inputs=[msg, chatbot],
        outputs=[msg, chatbot],
        queue=False,
    ).then(
        fn=bot,
        inputs=[
            chatbot,
            temperature,
            top_p,
            top_k,
            repetition_penalty,
            conversation_id,
        ],
        outputs=chatbot,
        queue=True,
    )
    
    stop.click(
        fn=request_cancel,
        inputs=None,
        outputs=None,
        cancels=[submit_event, submit_click_event],
        queue=False,
    )
    clear.click(lambda: None, None, chatbot, queue=False)
    
    gr.Markdown("<div style='text-align: center;'><span style='color: #1E90FF; font-size: larger; font-weight: bold;'>Created by Team Jithendhra</span></div>")
    gr.Markdown("""
    <div style="display: flex; justify-content: center;">
        <a href="https://in.linkedin.com/in/poluru-sai-chaithanya-b5b174270" style="color: black; font-size: small; font-weight: bold; text-align: center; margin: 0 10px;">
            Poluru Sai Chaitanya <img src="https://upload.wikimedia.org/wikipedia/commons/0/01/LinkedIn_Logo.svg" width="50" height="50" style="vertical-align: middle;">
        </a>
        <a href="https://in.linkedin.com/in/ramireddy-yaswanth-reddy" style="color: black; font-size: small; font-weight: bold; text-align: center; margin: 0 10px;">
            Ramireddygari Yaswanth Reddy <img src="https://upload.wikimedia.org/wikipedia/commons/0/01/LinkedIn_Logo.svg" width="50" height="50" style="vertical-align: middle;">
        </a>
        <a href="https://www.linkedin.com/in/jithendhralalam/" style="color: black; font-size: small; font-weight: bold; text-align: center; margin: 0 10px;">
            Lalam Jithendhra <img src="https://upload.wikimedia.org/wikipedia/commons/0/01/LinkedIn_Logo.svg" width="50" height="50" style="vertical-align: middle;">
        </a>
    </div>
""")

    demo.launch()

Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.
